In [1]:
import pandas as pd

df = pd.read_csv("./ds_salaries.csv")

display(df)

print(df["company_size"].unique())

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
3752,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


['L' 'S' 'M']


In [2]:
df_features = df.copy(deep=True)

target = df_features["salary_in_usd"]
df_features.drop(["salary", "salary_in_usd"], axis=1, inplace=True)

display(df_features)

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,EUR,ES,100,ES,L
1,2023,MI,CT,ML Engineer,USD,US,100,US,S
2,2023,MI,CT,ML Engineer,USD,US,100,US,S
3,2023,SE,FT,Data Scientist,USD,CA,100,CA,M
4,2023,SE,FT,Data Scientist,USD,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,USD,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,USD,US,100,US,L
3752,2020,EN,FT,Data Scientist,USD,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,USD,US,100,US,L


# Create preprocessor

In [3]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

oneHot_features = ["employment_type", "remote_ratio", "job_title", "salary_currency", "employee_residence", "company_location","company_size"]
ordinal_features = ["experience_level", "work_year"]
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(), oneHot_features),
    ('ord', OrdinalEncoder(categories=[
        ['EN', 'MI', 'SE', 'EX'],
        [2020,2021,2022,2023]
    ]), ordinal_features)])


df_preprocessed1 = preprocessor.fit_transform(df_features)



In [4]:

oneHot_features = ["work_year", "employment_type", "remote_ratio", "job_title", "salary_currency", "employee_residence", "company_location","company_size"]
ordinal_features = ["experience_level"]
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(), oneHot_features),
    ('ord', OrdinalEncoder(categories=[
        ['EN', 'MI', 'SE', 'EX'],
    ]), ordinal_features)])

df_preprocessed2 = preprocessor.fit_transform(df_features)



In [5]:

oneHot_features = ["experience_level", "employment_type", "remote_ratio", "job_title", "salary_currency", "employee_residence", "company_location","company_size"]
ordinal_features = ["work_year"]
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(), oneHot_features),
    ('ord', OrdinalEncoder(categories=[
        [2020, 2021, 2022, 2023],
    ]), ordinal_features)])

#preprocessor.set_output(transform="pandas")
df_preprocessed3 = preprocessor.fit_transform(df_features)
#display(df_preprocessed)


In [6]:

oneHot_features = ["work_year", "employment_type", "remote_ratio", "job_title", "salary_currency", "employee_residence", "company_location","company_size", "experience_level"]
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(), oneHot_features),
    ])
df_preprocessed4 = preprocessor.fit_transform(df_features)


# init Model

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

estimator = LinearRegression()

# Execute pipeline and evaluate

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

steps = [('estimator', estimator)]

pipeline = Pipeline(steps)

kfold = KFold(n_splits=5, shuffle=True, random_state = 42)

r2_scores = cross_val_score(pipeline, df_preprocessed1, target, cv=kfold, scoring='r2')
mse_scores = -cross_val_score(pipeline, df_preprocessed1, target, cv=kfold, scoring='neg_mean_squared_error')

r2_scores1 = cross_val_score(pipeline, df_preprocessed2, target, cv=kfold, scoring='r2')
mse_scores1 = -cross_val_score(pipeline, df_preprocessed2, target, cv=kfold, scoring='neg_mean_squared_error')

r2_scores2 = cross_val_score(pipeline, df_preprocessed3, target, cv=kfold, scoring='r2')
mse_scores2 = -cross_val_score(pipeline, df_preprocessed3, target, cv=kfold, scoring='neg_mean_squared_error')

r2_scores3 = cross_val_score(pipeline, df_preprocessed4, target, cv=kfold, scoring='r2')
mse_scores3 = -cross_val_score(pipeline, df_preprocessed4, target, cv=kfold, scoring='neg_mean_squared_error')

print("Variante1")
print(np.mean(r2_scores))
print(np.mean(mse_scores))
print("Variante2")
print(np.mean(r2_scores1))
print(np.mean(mse_scores1))
print("Variante3")
print(np.mean(r2_scores2))
print(np.mean(mse_scores2))
print("Variante4")
print(np.mean(r2_scores3))
print(np.mean(mse_scores3))

Variante1
0.3959480275066372
2399807110.378286
Variante2
0.3976433206856284
2393047675.412111
Variante3
0.39667088639226267
2396873912.7290926
Variante4
0.3982418303409419
2390607959.3154135


# init Model

In [12]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

estimator = GradientBoostingRegressor()

# Execute pipeline and evaluate

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

steps = [('estimator', estimator)]

pipeline = Pipeline(steps)

kfold = KFold(n_splits=5, shuffle=True, random_state = 42)

r2_scores = cross_val_score(pipeline, df_preprocessed4, target, cv=kfold, scoring='r2')
mse_scores = -cross_val_score(pipeline, df_preprocessed4, target, cv=kfold, scoring='neg_mean_squared_error')


print(np.mean(r2_scores))
print(np.mean(mse_scores))

0.420236616416464
2301740532.800516


In [11]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold 

# Modell initialisieren
regressor = GradientBoostingRegressor()

# Parameter Grid für Grid Search definieren
parameters = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [2, 3, 5]
}

k_10_fold_cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Grid Search initialisieren
grid_search_estimator = GridSearchCV(regressor, parameters, scoring='r2', cv=k_10_fold_cv, return_train_score=False)

grid_search_estimator.fit(df_preprocessed4,target)

#results = pd.DataFrame(grid_search_estimator.cv_results_)
    
# print the best parameter setting
print("best score is {} with params {}".format(grid_search_estimator.best_score_, grid_search_estimator.best_params_))


best score is 0.4331969080281256 with params {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 1000}


In [15]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold 

# Modell initialisieren
regressor = GradientBoostingRegressor()

# Parameter Grid für Grid Search definieren
parameters = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [2, 3, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

k_10_fold_cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Grid Search initialisieren
grid_search_estimator = GridSearchCV(regressor, parameters, scoring='r2', cv=k_10_fold_cv, n_jobs=-1, return_train_score=False)

grid_search_estimator.fit(df_preprocessed4,target)

#results = pd.DataFrame(grid_search_estimator.cv_results_)
    
# print the best parameter setting
print("best score is {} with params {}".format(grid_search_estimator.best_score_, grid_search_estimator.best_params_))


best score is 0.43718198300279154 with params {'learning_rate': 0.1, 'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}


In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold 

# Modell initialisieren
regressor = GradientBoostingRegressor()

# Parameter Grid für Grid Search definieren
parameters = {
    'learning_rate': [0.025, 0.05, 0.1, 0.2, 0.3],
    'max_depth': [2, 3, 5, 7, 10, 3800],
    'min_samples_split': [2, 5, 10, 20],
    'max_features': [0.25, 1.0, 'sqrt', 'log2'],
    'subsample': [0.15, 0.5, 0.75, 1.0]
}

k_10_fold_cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Grid Search initialisieren
grid_search_estimator = GridSearchCV(regressor, parameters, scoring='r2', cv=k_10_fold_cv, n_jobs=-1, return_train_score=False)

grid_search_estimator.fit(df_preprocessed4,target)

#results = pd.DataFrame(grid_search_estimator.cv_results_)
    
# print the best parameter setting
print("best score is {} with params {}".format(grid_search_estimator.best_score_, grid_search_estimator.best_params_))

best score is 0.4379085157268688 with params {'learning_rate': 0.1, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 20, 'subsample': 1.0}


In [9]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold 

# Modell initialisieren
regressor = GradientBoostingRegressor()

# Parameter Grid für Grid Search definieren
parameters = {
    'learning_rate': [0.1],
    'max_depth': [10],
    'min_samples_split': [20],
    'max_features': ['sqrt', 'log2'],
    'subsample': [1.0]
}

k_10_fold_cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Grid Search initialisieren
grid_search_estimator = GridSearchCV(regressor, parameters, scoring='r2', cv=k_10_fold_cv, return_train_score=False)

grid_search_estimator.fit(df_preprocessed4,target)

#results = pd.DataFrame(grid_search_estimator.cv_results_)
    
# print the best parameter setting
print("best score is {} with params {}".format(grid_search_estimator.best_score_, grid_search_estimator.best_params_))

best score is 0.43817034079751876 with params {'learning_rate': 0.1, 'max_depth': 10, 'max_features': 'log2', 'min_samples_split': 20, 'subsample': 1.0}
